## Requirements

코드 실행에 필요한 라이브러리를 사전에 설치합니다.

In [ ]:
!pip install transformers
!pip install datasets
!pip install peft
!pip install fire
!pip install accelerate
!pip install bitsandbytes
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=1169

## 라이브러리 불러오기

코드 실행에 필요한 라이브러리를 불러옵니다.

In [ ]:
import os
import sys
import textwrap
from typing import List

import datasets
import fire
import torch
import transformers
from datasets import load_dataset, Dataset, DatasetDict
from peft import (LoraConfig, PeftType, PromptTuningConfig, PromptTuningInit,
                  TaskType, get_peft_config, get_peft_model,
                  get_peft_model_state_dict, prepare_model_for_int8_training)
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          LlamaForCausalLM, LlamaTokenizer,
                          default_data_collator,
                          get_linear_schedule_with_warmup)


## 하이퍼파라미터 정의

모델 훈련에 사용할 하이퍼파라미터를 정의합니다.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
max_length = 64
learning_rate = 3e-2
num_epochs = 5  # 50
batch_size = 8

## 데이터 불러오기

구글 드라이브를 마운트하여 모델 학습, 검증 및 추론에 사용할 데이터를 불러옵니다.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pandas as pd

data_dir = "/content/drive/MyDrive/Colab Notebooks/Data"
raw_train_data = pd.read_csv(os.path.join(data_dir, "train_data_frac.csv"))
raw_val_data = pd.read_csv(os.path.join(data_dir, "val_data_frac.csv"))
raw_test_data = pd.read_csv(os.path.join(data_dir, "test_data_frac.csv"))

In [ ]:
raw_train_data.head(5)

,question,context,answer,other_answers
0,아르헨티나 헌법 전문에 의해 아르헨티나 국민 대표들은 기존 협약을 이행함에 있어 무...,전 문 우리는 아르헨티나 국민의 대표로서 각 주의 뜻과 선거에 의해 연방제헌의회에 ...,"국가를 통합하고, 정의를 실현하고, 국내평화를 지키고, 공동방위를 제공하고, 사회복...",NaN
1,벤 애플렉이 맷 데이먼과 공동으로 설립한 제작사는?,"2000년 데이먼은 애플렉, 크리스 무어, 숀 베일리와 함께 영화 제작사 라이브플래...",라이브플래닛,NaN
2,중앙대학교 연극영화학과와 중앙대학교 예술대학원을 나온 박상아와 남자 프로레슬링선수이...,박상아. 생애. 본관은 밀양(密陽)이며 신장은 166cm이고 체중은 47kg인 그녀...,no,NaN
3,네마냐 비디치는 어느 나라의 축구 선수 였는가?,- 네마냐 비디치: 세르비아의 전 축구 선수,세르비아,NaN
4,국회 프락치 사건은 어떤 의원이 체포된 사건인가?,"국회 프락치 사건은 1949년 6월, 이른바 '남로당 프락치(공작원)'로 제헌국회에...",김약수 의원,NaN


## 모델 정의

LLaMA 모델을 정의합니다.

In [ ]:
BASE_MODEL = "decapoda-research/llama-7b-hf"

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

model.config.use_cache = False

tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
def generate_prompt(data_point):
    return f"""아래는 작업을 설명하는 명령어입니다. 문맥에 맞게 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어: {data_point["instruction"]} ### 문맥: {data_point["input"]} ### 응답: {data_point["output"]}"""


def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=1024,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < 1024
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt


In [ ]:
def make_dataset_from_df(raw_df):
    raw_df = raw_df.rename(columns={
        "question": "instruction",
        "context": "input",
        "answer": "output"
        })

    dataset = datasets.Dataset.from_pandas(raw_df)
    return dataset

train_dataset = make_dataset_from_df(raw_train_data)
val_dataset = make_dataset_from_df(raw_val_data)
test_dataset = make_dataset_from_df(raw_test_data)

In [ ]:
generate_prompt(train_dataset[0])

'아래는 작업을 설명하는 명령어입니다. 문맥에 맞게 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어: 아르헨티나 헌법 전문에 의해 아르헨티나 국민 대표들은 기존 협약을 이행함에 있어 무엇을 위해 동법을 제정하였는가? ### 문맥: 전 문 우리는 아르헨티나 국민의 대표로서 각 주의 뜻과 선거에 의해 연방제헌의회에 모여 기존 협약을 이행함에 있어, 국가를 통합하고, 정의를 실현하고, 국내평화를 지키고, 공동방위를 제공하고, 사회복지를 증진하고, 자유의 축복을 우리와 우리 후손 그리고 아르헨티나 땅에 살기를 원하는 전 세계인에게 보장하기 위해, 모든 이성과 정의의 원천인 신의 가호를 기원하면서 신의 뜻에 따라 아르헨티나를 위한 이 헌법을 정하고, 명하고, 제정한다. ### 응답: 국가를 통합하고, 정의를 실현하고, 국내평화를 지키고, 공동방위를 제공하고, 사회복지를 증진하고, 자유의 축복을 우리와 우리 후손 그리고 아르헨티나 땅에 살기를 원하는 전 세계인에게 보장하기 위해'

In [ ]:
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [ ]:
generate_prompt(dataset["train"][0])

'아래는 작업을 설명하는 명령어입니다. 문맥에 맞게 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어: 아르헨티나 헌법 전문에 의해 아르헨티나 국민 대표들은 기존 협약을 이행함에 있어 무엇을 위해 동법을 제정하였는가? ### 문맥: 전 문 우리는 아르헨티나 국민의 대표로서 각 주의 뜻과 선거에 의해 연방제헌의회에 모여 기존 협약을 이행함에 있어, 국가를 통합하고, 정의를 실현하고, 국내평화를 지키고, 공동방위를 제공하고, 사회복지를 증진하고, 자유의 축복을 우리와 우리 후손 그리고 아르헨티나 땅에 살기를 원하는 전 세계인에게 보장하기 위해, 모든 이성과 정의의 원천인 신의 가호를 기원하면서 신의 뜻에 따라 아르헨티나를 위한 이 헌법을 정하고, 명하고, 제정한다. ### 응답: 국가를 통합하고, 정의를 실현하고, 국내평화를 지키고, 공동방위를 제공하고, 사회복지를 증진하고, 자유의 축복을 우리와 우리 후손 그리고 아르헨티나 땅에 살기를 원하는 전 세계인에게 보장하기 위해'

In [ ]:
train_data = dataset["train"].map(generate_and_tokenize_prompt)
val_data = dataset["validation"].map(generate_and_tokenize_prompt)

Map:   0%|          | 0/4745 [00:00<?, ? examples/s]

Map:   0%|          | 0/949 [00:00<?, ? examples/s]

In [ ]:
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

BATCH_SIZE = 16
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 300
OUTPUT_DIR = "experiments"

In [ ]:
model = prepare_model_for_int8_training(model)
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)
model.print_trainable_parameters()

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.06220594176090199


In [ ]:
training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=100,
    max_steps=TRAIN_STEPS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=10,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=50,
    save_steps=50,
    output_dir=OUTPUT_DIR,
    save_total_limit=3,
    load_best_model_at_end=True,
    report_to="tensorboard"
)

In [ ]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    args=training_arguments,
    data_collator=data_collator
)
model.config.use_cache = False
old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))

model = torch.compile(model)

# trainer.train()
# model.save_pretrained(OUTPUT_DIR)

In [ ]:
trainer.train()
model.save_pretrained(OUTPUT_DIR)

/usr/local/lib/python3.10/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss


OutOfMemoryError: ignored